In [81]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import *
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import *
from  scipy import stats
from sklearn.model_selection import cross_val_score
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
import math


In [82]:
encoder = TargetEncoder()
weatheDf = pd.read_csv("weather.csv")
weatheDf["encodedlocation"] = encoder.fit_transform(weatheDf['location'], weatheDf['rain'])
weatheDf.time_stamp = pd.to_datetime(weatheDf.time_stamp,unit='s')


In [83]:
#weatheDf.month = weatheDf.time_stamp.dt.month
weatheDf['month'] = weatheDf.time_stamp.dt.month
weatheDf.time_stamp = weatheDf.time_stamp.dt.hour
weatheDf =  weatheDf.rename(
    columns ={
        'time_stamp':'hour'
    }
    )

In [84]:
scaler = MinMaxScaler()
scaler.fit(weatheDf.drop(['rain','location'],axis=1))
#wweatheDf = pd.DataFrame(scaler.transform(weatheDf.drop('rain',axis=1)), columns= weatheDf.drop('rain',axis=1).columns)
weatheDf1 =weatheDf.copy()
weatheDf.dropna(axis=0,inplace=True)
X = weatheDf.drop(['rain','location'],axis=1)
y = weatheDf.rain
x_pred = weatheDf1.drop(['rain','location'],axis=1)

#### Train the model to predict the missing values of rain feature

In [85]:
model = linear_model.LinearRegression()
poly_features = PolynomialFeatures(degree=3)
X_train_poly = poly_features.fit_transform(X)
x_pred_poly = poly_features.fit_transform(x_pred)
model.fit(X_train_poly, y)
y_train_predicted = model.predict(x_pred_poly)


In [86]:
y_train_predicted.shape
y_train_predicted
weatheDf1.rain = y_train_predicted


In [87]:
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.8,frac_test=0.2,
                                         random_state=None):
    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

 

    assert len(df_input) == len(df_train) + len(df_temp)

    return df_train, df_temp

In [88]:
def Feature_Encoder(X, cols):
    for c in cols:
        lbl = LabelEncoder()
        lbl.fit(list(X[c].values))
        X[c] = lbl.transform(list(X[c].values))
    return X


In [89]:
def one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df


In [90]:
weatheDf1.columns

Index(['temp', 'location', 'clouds', 'pressure', 'rain', 'hour', 'humidity',
       'wind', 'encodedlocation', 'month'],
      dtype='object')

## working on the Main Dataset

In [91]:
weatherData =weatheDf1.drop('encodedlocation',axis=1)
taxiRidersData = pd.read_csv("taxi-rides-classification.csv")
taxiRidersData.time_stamp = pd.to_datetime(taxiRidersData.time_stamp,unit="ms")
#taxiRidersData.month =taxiRidersData.time_stamp.dt.month
taxiRidersData['month']=(taxiRidersData.time_stamp.dt.month)
taxiRidersData.time_stamp = taxiRidersData.time_stamp.dt.hour
taxiRidersData.rename(
    columns ={
        'time_stamp':'hour'
    }
)


distance cab_type  hour         destination                   source  \
0           0.62     Uber     6            West End         Haymarket Square   
1           2.27     Uber    10   Boston University              Beacon Hill   
2           2.00     Lyft     0            Back Bay         Haymarket Square   
3           3.98     Lyft    23  Financial District  Northeastern University   
4           1.49     Lyft    12            Back Bay  Northeastern University   
...          ...      ...   ...                 ...                      ...   
554451      1.64     Uber    23            Back Bay  Northeastern University   
554452      0.92     Uber     3       North Station                North End   
554453      2.66     Uber    22    Theatre District                   Fenway   
554454      2.98     Uber     7    Theatre District        Boston University   
554455      1.91     Uber    23    Theatre District  Northeastern University   

        surge_multiplier                                    id  \
0                    1.0  c1b4a572-24b2-484d-aac5-b4f97b790480   
1                    1.0  f9e7e7e6-b309-479b-9f90-bf26a1870c81   
2                    1.0  154e8438-46f5-4e5f-9872-15cc95e85eed   
3                    1.0  6bdc30a6-9a80-487b-914e-8d9b82ee4acb   
4                    1.0  0cb12fe9-cd72-4feb-b341-4a566b2e6643   
...                  ...                                   ...   
554451               1.0  2aa98e8a-4f06-4ae2-9291-036d74094a1b   
554452               1.0  75cb7a6d-68be-4cc7-b4da-c8e4b997aa8a   
554453               1.0  569da280-71c6-4459-a343-8573f8613415   
554454               1.0  71d8a3a6-5b0e-4710-a803-dfd5680a590a   
554455               1.0  cd258703-5a56-4d87-b9c3-5166210698ea   

                                  product_id       name RideCategory  month  
0       8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a       Taxi      unknown     12  
1       997acbb5-e102-41e1-b155-9df7de0a73f2   UberPool        cheap     11  
2                                       lyft       Lyft     moderate     12  
3                                  lyft_plus    Lyft XL    expensive     11  
4                                       lyft       Lyft        cheap     11  
...                                      ...        ...          ...    ...  
554451  6f72dfc5-27f1-42e8-84db-ccc7a75f6969     UberXL     moderate     11  
554452  6d318bcc-22a3-4af6-bddd-b409bfce1546  Black SUV    expensive     11  
554453  6f72dfc5-27f1-42e8-84db-ccc7a75f6969     UberXL     moderate     11  
554454  55c66225-fbe7-4fd5-9072-eab1ece5e23e      UberX     moderate     12  
554455  6f72dfc5-27f1-42e8-84db-ccc7a75f6969     UberXL     moderate     11  

[554456 rows x 11 columns]

In [92]:
taxiRidersData.dropna(axis=0, inplace=True)
avgWeather = weatherData.groupby(["location"]).mean().reset_index(drop=False)
print(taxiRidersData.columns)
avgWeather.drop(['hour'],axis=1,inplace=True)
avgWeather.drop(['month'],axis=1,inplace=True)
sourceWeather = avgWeather.rename(
    columns={
        'location': 'source',
        'rain': 'source_rain',
        'temp': 'source_temp',
        'clouds': 'source_clouds',
        'pressure': 'source_pressure',
        'humidity': 'source_humidity',
        'wind': 'source_wind'
    }
)

destinationWeather = avgWeather.rename(
    columns={
        'location': 'destination',
        'rain': 'destination_rain',
        'temp': 'destination_temp',
        'clouds': 'destination_clouds',
        'pressure': 'destination_pressure',
        'humidity': 'destination_humidity',
        'wind': 'destination_wind'

    }
)


Index(['distance', 'cab_type', 'time_stamp', 'destination', 'source',
       'surge_multiplier', 'id', 'product_id', 'name', 'RideCategory',
       'month'],
      dtype='object')


In [93]:
data = taxiRidersData.merge(sourceWeather, on='source')\
    .merge(destinationWeather, on="destination")
print(data.columns)

Index(['distance', 'cab_type', 'time_stamp', 'destination', 'source',
       'surge_multiplier', 'id', 'product_id', 'name', 'RideCategory', 'month',
       'source_temp', 'source_clouds', 'source_pressure', 'source_rain',
       'source_humidity', 'source_wind', 'destination_temp',
       'destination_clouds', 'destination_pressure', 'destination_rain',
       'destination_humidity', 'destination_wind'],
      dtype='object')


### DATA PRE-PROCESSING


In [94]:
# for i  in ((data.product_id + " " + data.name).unique()):
#    print(i)
data.drop(['product_id'],axis=1,inplace=True)
data.drop(['id'],axis=1,inplace=True)
print(data.columns)

Index(['distance', 'cab_type', 'time_stamp', 'destination', 'source',
       'surge_multiplier', 'name', 'RideCategory', 'month', 'source_temp',
       'source_clouds', 'source_pressure', 'source_rain', 'source_humidity',
       'source_wind', 'destination_temp', 'destination_clouds',
       'destination_pressure', 'destination_rain', 'destination_humidity',
       'destination_wind'],
      dtype='object')


In [95]:
corr = data.corr()
plt.figure(figsize=(16, 5))
dataplot = sns.heatmap(data.corr(), annot=True, linewidths=1)

In [96]:
dataX = data.drop(['RideCategory'],axis=1)
cor_matrix = dataX.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.92)]
print(to_drop)
data = data.drop(to_drop, axis=1)

['source_humidity', 'source_wind', 'destination_humidity', 'destination_wind']


### remove outliers

In [97]:
def cap_data(df):
    for col in df.columns:
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.25,0.75]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

In [98]:
data.RideCategory.unique()

array(['unknown', 'moderate', 'cheap', 'expensive', 'very expensive'],
      dtype=object)

### Encdoing Data

In [99]:
lables = {'unknown':0, 'moderate':2, 'cheap':1, 'expensive':3, 'very expensive':4}
data['RideCategory']=data['RideCategory'].map(lambda x: lables[x])
dataTrain,dataTest = split_stratified_into_train_val_test(data,stratify_colname='RideCategory', frac_train=0.60, frac_test=0.20)

In [100]:
dataTrain = cap_data(dataTrain)
dataTest = cap_data(dataTest)

/tmp/ipykernel_62859/2223872166.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df[col] <= percentiles[0]] = percentiles[0]
/tmp/ipykernel_62859/2223872166.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df[col] >= percentiles[1]] = percentiles[1]
/tmp/ipykernel_62859/2223872166.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df[col] <= percentiles[0]] = percentiles[0]
/tmp/ipykernel_62859/2223872166

Using mean Encdoing to encode data since it has great balance between efficency and model complexity

In [101]:
def mean_encoding(data):
    for col in data.select_dtypes(include='O').columns:
        encoder = TargetEncoder()
        data[col] = encoder.fit_transform(data[col],data['RideCategory'])
    return data


In [102]:

dataTrain = mean_encoding(dataTrain)
dataTest = mean_encoding(dataTest)

In [103]:
corr = dataTest.corr()
plt.figure(figsize=(16, 5))
dataplot = sns.heatmap(corr, annot=True, linewidths=1)
top_feature = corr.index[abs(corr['RideCategory']) > 0.2]
print(top_feature)
dataTest = dataTest[top_feature]
dataTrain = dataTrain[top_feature]


Index(['distance', 'name', 'RideCategory'], dtype='object')


In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=10)
X_train = dataTrain.drop(['RideCategory'],axis=1)
X_test = dataTest.drop(['RideCategory'],axis=1)
y_train = dataTrain.RideCategory
y_test = dataTest.RideCategory

NameError: name 'Y' is not defined

In [108]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns= dataTrain.drop(['RideCategory'],axis=1).columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns= dataTrain.drop(['RideCategory'],axis=1).columns)


In [109]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [111]:
pred = clf.predict(X_test)
print('model Test Mean Square Error', metrics.mean_squared_error(y_test, pred))

model Test Mean Square Error 0.10823642930251642


In [112]:
print('model Test R2', metrics.r2_score(y_test, pred))

model Test R2 0.8403402626053508
